## Import libraries
The first step is to import the libraries required to query the data in question. To do this, run the below cell.

In [1]:
from vortexasdk import CargoMovements, CargoTimeSeries, OnshoreInventoriesTimeseries, Geographies, Products, StorageTerminals
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import time
import plotly.express as px
import dateutil.relativedelta

## Search for IDs
In the Vortexa SDK, we cannot refer to products, vessels or geographies by name. Each product, vessel or geography in our data has its own unique ID. These IDs are used to refer to the products, vessels or geographies in our database when making queries.

Once you find the ID, you can copy it and assign it as an object. This way, you can refer to the product or geography by the name you have given to it. Some examples have been done below.

In [2]:
# search for geography ids (remove hashtags to search)

# full_length_df = Geographies().search(term=["United States"]).to_df()
# print(full_length_df.to_string(index=False))

# full_length_df = StorageTerminals().search(term=["Cushing"]).to_df()
# print(full_length_df.to_string(index=False))

# Store geography ids

united_states='2d92cc08f22524dba59f6a7e340f132a9da0ce9573cca968eb8e3752ef17a963'
cushing='cde783a902c7837b814dfa5988ed74fb14841c2999d6984952c7cbfc543d5073'

# search for product ids (remove hashtags below to search)

# product_search = Products().search(term=['Crude']).to_df()
# print (product_search.to_string(index=False))

# Store product ids
crude='54af755a090118dcf9b0724c9a4e9f14745c26165385ffa7f1445bc768f06f11'

# Main functions
Run the cell below to define the required functions.

In [3]:
# Weekly crude storage function
def onshore_crude_inventories_time_series(start_y, start_m, start_d, locs, unit, freq):
    
    # Define constants
    today=datetime.today() # - dateutil.relativedelta.relativedelta(days=1)
    
    # Pull onshore crude inventory data
    inventories = OnshoreInventoriesTimeseries().search(
        time_min=datetime(start_y, start_m, start_d),
        time_max=today,
        crude_confidence=['confirmed', 'probable'],
        location_ids=locs,
        timeseries_frequency=freq,
        timeseries_unit=unit,
        timeseries_unit_operator="fill").to_df()

    # Convert dates to weeks and years for merging
    inventories["weeks"] = inventories['key'].dt.strftime('W%U-%Y')
    inventories=pd.concat([inventories['key'], inventories['value']], axis=1)
    inventories.columns=['Date', 'Crude inventories']
    
    return inventories

def eia_weekly_change(year=datetime.today().year, month=datetime.today().month, day=datetime.today().day, region='US'):    
    
    # define constants
    if region=='US':
        
        loc='2d92cc08f22524dba59f6a7e340f132a9da0ce9573cca968eb8e3752ef17a963'
        
    elif region=='Cushing':
        
        loc='cde783a902c7837b814dfa5988ed74fb14841c2999d6984952c7cbfc543d5073'

    current_date=datetime(year, month, day)
    current_year=current_date.year
    current_month=current_date.month
    last_month=(current_date-dateutil.relativedelta.relativedelta(months=1)).month

    current_day=current_date.strftime('%a')

    # pull weekly YTD inventories data
    result=onshore_crude_inventories_time_series(start_y=current_year, start_m=last_month, start_d=1, 
                                                 locs=[loc], unit='b', freq='day')

    result['DoW']=result['Date'].dt.strftime('%a')

    # Calculate last Friday
    if current_date.weekday() == 4:  # If today is Friday
        last_friday = current_date - timedelta(days=7)
    else:
        last_friday = current_date - timedelta(days=(current_date.weekday() - 4 + 7) % 7)

    last_friday = datetime(last_friday.year, last_friday.month, last_friday.day)
    friday_before_last=last_friday - timedelta(days=7)

    # Calculate next Friday
    if current_date.weekday() == 4:  # If today is Friday
        next_friday = current_date
    else:
        next_friday = current_date + timedelta(days=(4 - current_date.weekday() + 7) % 7)

    next_friday = datetime(next_friday.year, next_friday.month, next_friday.day)

    # Calculate last Wednesay
    if current_date.weekday() == 2:  # If today is Wednesday
        last_wednesday = current_date - timedelta(days=7)
    else:
        last_wednesday = current_date - timedelta(days=(current_date.weekday() - 2 + 7) % 7)

    last_wednesday = datetime(last_wednesday.year, last_wednesday.month, last_wednesday.day)

    # Calculate next Wednesday
    if current_date.weekday() == 2:  # If today is Wednesday
        next_wednesday = current_date
    else:
        next_wednesday = current_date + timedelta(days=(2 - current_date.weekday() + 7) % 7)

    next_wednesday = datetime(next_wednesday.year, next_wednesday.month, next_wednesday.day)
    wednesday_after_next=next_wednesday + timedelta(days=7)

    result['Date'] = pd.to_datetime(result['Date']).dt.date

    friday_before_last = friday_before_last.date()
    last_friday = last_friday.date()
    next_friday = next_friday.date()

    last_wednesday = last_wednesday.date()
    next_wednesday = next_wednesday.date()
    wednesday_after_next = wednesday_after_next.date()

    today=current_date.date()

    if current_day in ['Fri']:

        next_friday_value=result[result['Date']==next_friday]['Crude inventories'].values[0]
        last_friday_value=result[result['Date']==last_friday]['Crude inventories'].values[0]

        eia_change=(next_friday_value-last_friday_value) / 1000000.0

        if eia_change > 0:
            direction1='build'

        elif eia_change < 0:
            direction1='draw'

        elif eia_change==0:
            direction1='flat'

        eia_change_abs=abs(eia_change)

        print(f"Vortexa estimate for EIA stock change ({region})", 
              f"on Wednesday {next_wednesday.day}", 
              f"{next_wednesday.strftime('%b')}", 
              f"{next_wednesday.year}: ", 
              f"{eia_change_abs} mbbl", 
              f"{direction1}")
        
        print(' ')
        
        print(f"Vortexa estimate for EIA stock change ({region})", 
              f"on Wednesday {wednesday_after_next.day}", 
              f"{wednesday_after_next.strftime('%b')}", 
              f"{wednesday_after_next.year} available from Saturday")

    elif current_day in['Sat', 'Sun', 'Mon', 'Tue', 'Wed']:

        next_friday_value=result[result['Date']==today]['Crude inventories'].values[0]
        last_friday_value=result[result['Date']==last_friday]['Crude inventories'].values[0]
        friday_before_last_value=result[result['Date']==friday_before_last]['Crude inventories'].values[0]

        eia_change=(last_friday_value-friday_before_last_value) / 1000000.0

        if eia_change > 0:
            direction1='build'

        elif eia_change < 0:
            direction1='draw'

        elif eia_change==0:
            direction1='flat'

        eia_change_abs=abs(eia_change)

        print(f"Vortexa estimate for EIA stock change ({region})", 
              f"on Wednesday {next_wednesday.day}", 
              f"{next_wednesday.strftime('%b')}", 
              f"{next_wednesday.year}: ", 
              f"{eia_change_abs} mbbl", 
              f"{direction1}")
        print(" ")

        next_eia_change=(next_friday_value-last_friday_value) / 1000000.0

        if next_eia_change > 0:
            direction2='build'

        elif next_eia_change < 0:
            direction2='draw'

        elif next_eia_change==0:
            direction2='flat'

        next_eia_change_abs=abs(next_eia_change)

        print(f"Preliminary Vortexa estimate for EIA stock change ({region})", 
              f"on Wednesday {wednesday_after_next.day}", 
              f"{wednesday_after_next.strftime('%b')}", 
              f"{wednesday_after_next.year}: ", 
              f"{next_eia_change_abs} mbbl", 
              f"{direction2}") 
        
    elif current_day in ['Thu']:
        
        next_friday_value=result[result['Date']==today]['Crude inventories'].values[0]
        last_friday_value=result[result['Date']==last_friday]['Crude inventories'].values[0]
        
        next_eia_change=(next_friday_value-last_friday_value) / 1000000.0

        if next_eia_change > 0:
            direction2='build'

        elif next_eia_change < 0:
            direction2='draw'

        elif next_eia_change==0:
            direction2='flat'

        next_eia_change_abs=abs(next_eia_change)

        print(f"Preliminary Vortexa estimate for EIA stock change ({region})", 
              f"on Wednesday {next_wednesday.day}", 
              f"{next_wednesday.strftime('%b')}", 
              f"{next_wednesday.year}: ", 
              f"{next_eia_change_abs} mbbl", 
              f"{direction2}") 
        
        print(' ')
        
        print(f"Vortexa estimate for EIA stock change ({region})", 
              f"on Wednesday {wednesday_after_next.day}", 
              f"{wednesday_after_next.strftime('%b')}", 
              f"{wednesday_after_next.year} available from Saturday")
        
        
    return result

## EIA stock change estimate
The code below produces a figure which should be the closest to what the EIA releases, typically on a Wednesday. 

Note, you can change the region argument to 'Cushing' to see Cushing stock change estimates. You can also specify the year, month and day where you would like to see historical estimates (the default is to show the latest estimates).

Run the code below to see estimate

In [4]:
eia_change=eia_weekly_change(region='US')

Vortexa estimate for EIA stock change (US) on Wednesday 18 Sep 2024:  1.648906 mbbl draw
 
Preliminary Vortexa estimate for EIA stock change (US) on Wednesday 25 Sep 2024:  0.990346 mbbl draw
